## Imports
Execute the cell below to import all packages needed.

In [1]:
# System functionality
import sys
sys.path.append('..')

import glob
import os
import numpy as np 
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import seaborn as sns
import graphviz
import math as math
import json
import numpy
import cv2
import scipy.stats as stats
import matplotlib.image as mpimg
import sklearn.feature_selection as feature_selection
from skimage import data, io, filters
from numpy import linspace
from numpy.core.umath import pi
from matplotlib.ticker import FuncFormatter
from matplotlib.font_manager import FontProperties
from emoji_extractor.extract import Extractor
from emojipedia import Emojipedia
from skimage import io
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import chi2_contingency
from scipy.interpolate import spline

## Load csv into pandas dataframe

In [3]:
path_counts_ran = '/home/stelios/Desktop/Honours Project/Samples/per_user_emoji_counts/london_per_user_emoji_counts.csv'
path_users_ran = '/home/stelios/Desktop/Honours Project/Samples/user_data/lon.csv'
counts_df_ran = pd.read_csv(path_counts_ran,index_col=None, header=0, low_memory = False)
users_df_ran = pd.read_csv(path_users_ran,index_col=None, header=0, low_memory = False)

In [4]:
counts_df = counts_df_ran 

# Fill NaN with 0.0
counts_df = counts_df.fillna(0.0)

In [5]:
# Feature % representation
for index, row in counts_df.iterrows():
    counts_df.loc[index] = counts_df.loc[index]/counts_df.loc[index].sum()
counts_df.describe()

,🧒🏽,👱🏾,👩🏽‍💼,👨🏼‍🔬,🍘,🏄🏿‍♂️,🧪,🇧🇼,👨🏻‍🚒,🤜🏿,...,🇺🇿,🇸🇲,🇻🇬,🇹🇨,🇲🇶,🇲🇻,🇧🇾,🇲🇷,🇮🇴,🇲🇱
count,7.014000e+03,7.014000e+03,7.014000e+03,7014.000000,7.014000e+03,7.014000e+03,7.014000e+03,7014.000000,7014.000000,7.014000e+03,...,7014.000000,7014.000000,7014.000000,7.014000e+03,7.014000e+03,7014.000000,7014.000000,7.014000e+03,7014.000000,7014.000000
mean,1.320111e-07,3.292656e-07,7.833626e-07,0.000002,8.736029e-08,3.960333e-07,3.960333e-07,0.000002,0.000011,1.136938e-07,...,0.000002,0.000003,0.000004,1.437218e-07,6.253158e-07,0.000003,0.000002,3.072672e-07,0.000004,0.000002
std,1.105588e-05,2.757587e-05,6.560633e-05,0.000128,7.316392e-06,3.316764e-05,3.316764e-05,0.000125,0.000918,9.521812e-06,...,0.000166,0.000248,0.000362,1.203664e-05,5.236996e-05,0.000271,0.000126,2.573352e-05,0.000362,0.000134
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000
max,9.259259e-04,2.309469e-03,5.494505e-03,0.010753,6.127451e-04,2.777778e-03,2.777778e-03,0.009901,0.076923,7.974482e-04,...,0.013889,0.020619,0.030303,1.008065e-03,4.385965e-03,0.022727,0.010309,2.155172e-03,0.030303,0.011236


In [6]:
counts_df['gender'] = users_df_ran['gender']
counts_df['ethnicity'] = users_df_ran['ethnicity']

In [7]:
counts_df = counts_df.dropna()

In [8]:
counts_df.groupby('gender').size()

gender
female    1870
male      1857
dtype: int64

In [9]:
counts_df.groupby('ethnicity').size()

ethnicity
asian         52
black        719
hispanic      43
other          7
white       2906
dtype: int64

In [10]:
## count again all 
counts_all_df = counts_df_ran 

# Fill NaN with 0.0
counts_all_df = counts_all_df.fillna(0.0)

# Feature % representation
for index, row in counts_all_df.iterrows():
    counts_all_df.loc[index] = counts_all_df.loc[index]/counts_all_df.loc[index].sum()
counts_all_df.describe()

,🧒🏽,👱🏾,👩🏽‍💼,👨🏼‍🔬,🍘,🏄🏿‍♂️,🧪,🇧🇼,👨🏻‍🚒,🤜🏿,...,🇺🇿,🇸🇲,🇻🇬,🇹🇨,🇲🇶,🇲🇻,🇧🇾,🇲🇷,🇮🇴,🇲🇱
count,7.014000e+03,7.014000e+03,7.014000e+03,7014.000000,7.014000e+03,7.014000e+03,7.014000e+03,7014.000000,7014.000000,7.014000e+03,...,7014.000000,7014.000000,7014.000000,7.014000e+03,7.014000e+03,7014.000000,7014.000000,7.014000e+03,7014.000000,7014.000000
mean,1.320111e-07,3.292656e-07,7.833626e-07,0.000002,8.736029e-08,3.960333e-07,3.960333e-07,0.000002,0.000011,1.136938e-07,...,0.000002,0.000003,0.000004,1.437218e-07,6.253158e-07,0.000003,0.000002,3.072672e-07,0.000004,0.000002
std,1.105588e-05,2.757587e-05,6.560633e-05,0.000128,7.316392e-06,3.316764e-05,3.316764e-05,0.000125,0.000918,9.521812e-06,...,0.000166,0.000248,0.000362,1.203664e-05,5.236996e-05,0.000271,0.000126,2.573352e-05,0.000362,0.000134
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000
max,9.259259e-04,2.309469e-03,5.494505e-03,0.010753,6.127451e-04,2.777778e-03,2.777778e-03,0.009901,0.076923,7.974482e-04,...,0.013889,0.020619,0.030303,1.008065e-03,4.385965e-03,0.022727,0.010309,2.155172e-03,0.030303,0.011236


In [11]:
counts_df = counts_df.dropna()

In [12]:
counts_df.shape

(3727, 2243)